# Writing Vector Data with Fiona

Let's assume we have the following data and we want to create a vector dataset using Fiona:

In [1]:
import fiona
from fiona.crs import from_epsg

In [2]:
data = [[45.922513343092916, 7.835574679184418,'Liskamm',4527],
[45.941997570720375, 7.869820276613906,'Nordend',4609],
[46.10902325837147, 7.863895545667632,'Nadelhorn',4327],
[45.932186337151684, 7.8714190183674555,'Zumsteinspitze',4563],
[46.08336532442726, 7.857296913890337,'Täschhorn',4491],
[45.91669904679932, 7.863563975062021,'Ludwigshöhe',4341],
[45.93756139078208, 7.299279971077615,'Grand Combin de Grafeneire',4314],
[45.922513343092916, 7.835574679184418,'Lyskamm',4527],
[45.93683662540408, 7.866814344981748,'Dufourspitze (Pointe Dufour)',4634],
[46.10129664518156, 7.716156885858494,'Weisshorn',4506],
[45.976340506120614, 7.658691510512221,'Monte Cervino',4478],
[45.976340506120614, 7.658691510512221,'Matterhorn',4478],
[45.93674004101607, 7.86855410887458,'Grenzgipfel',4618],
[45.92712756883081, 7.876921984235257,'Signalkuppe (Punta Gnifetti)',4554],
[46.093839189553464, 7.858928716434883,'Dom',4545],
[46.107109586833495, 7.711724522200983,'Grand Gendarme',4331],
[45.919638502715564, 7.8711910872756405,'Parrotspitze',4432],
[46.093839189553464, 7.858928716434883,'Mischabel',4545],
[46.03426257063022, 7.61204033560156,'Dent Blanche',4357]]

### Creating a Schema


In [3]:
mountain_peaks_schema = {
    'geometry': 'Point',
    
    'properties' : {
        "name": "str",
        "elevation": "float",
    },
}
    

### Defining our Coordinate Reference System

In [4]:
mountain_peaks_crs = from_epsg(4326) 

### Defining our Output Format

In [5]:
fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [6]:
driver = "GPKG"

### Creating File and Writing Data

In [7]:
with fiona.open("geodata/peaks_fiona.gpkg", "w", driver=driver, crs=mountain_peaks_crs, schema = mountain_peaks_schema) as c:
    for record in data:
        
        longitude = record[1]
        latitude = record[0]
        name = record[2]
        elevation = record[3]
        
        entry = {}
        entry['geometry'] = { 'coordinates': (longitude, latitude), 
                              'type': 'Point' }
        
        entry['properties'] = {'name': name, 
                               'elevation': elevation } 
        

        
        c.write(entry)

### Read our GPKG

We could open our GPKG with QGIS

or display all data in Python

In [8]:
layers = fiona.listlayers("geodata/peaks_fiona.gpkg")
layers

['peaks_fiona']

In [9]:
with fiona.open("geodata/peaks_fiona.gpkg", 'r') as c:
    for peak in c:
        props = dict(peak['properties'])
        geom = dict(peak['geometry'])
        id = peak['id']
        print(id, props, geom)

1 {'name': 'Liskamm', 'elevation': 4527.0} {'coordinates': (7.835574679184418, 45.922513343092916), 'type': 'Point', 'geometries': None}
2 {'name': 'Nordend', 'elevation': 4609.0} {'coordinates': (7.869820276613906, 45.941997570720375), 'type': 'Point', 'geometries': None}
3 {'name': 'Nadelhorn', 'elevation': 4327.0} {'coordinates': (7.863895545667632, 46.10902325837147), 'type': 'Point', 'geometries': None}
4 {'name': 'Zumsteinspitze', 'elevation': 4563.0} {'coordinates': (7.8714190183674555, 45.932186337151684), 'type': 'Point', 'geometries': None}
5 {'name': 'Täschhorn', 'elevation': 4491.0} {'coordinates': (7.857296913890337, 46.08336532442726), 'type': 'Point', 'geometries': None}
6 {'name': 'Ludwigshöhe', 'elevation': 4341.0} {'coordinates': (7.863563975062021, 45.91669904679932), 'type': 'Point', 'geometries': None}
7 {'name': 'Grand Combin de Grafeneire', 'elevation': 4314.0} {'coordinates': (7.299279971077615, 45.93756139078208), 'type': 'Point', 'geometries': None}
8 {'name':

In [10]:
c = fiona.open("geodata/peaks_fiona.gpkg", 'r') 
print(c.schema)
c.close()

{'properties': {'name': 'str', 'elevation': 'float'}, 'geometry': 'Point'}
